In [ ]:
!nvidia-smi

In [ ]:
#@title WANDB Login { run: "auto", display-mode: "form" }
wandbapi = "" #@param {type:"string"}
!pip install wandb > /dev/null
!wandb login "$wandbapi"
%env USE_WANDB=1

In [ ]:
%cd /content/INFOMCV/
!git pull origin master

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
#@title Load cached dataset from google drive
drive_project_root = "MyDrive/Colab/INFOCV" #@param {type:"string"}
dataset_name = "stanford40_augmented.tar.gz" #@param {type:"string"}
import os
from google.colab import drive
drive.mount('/content/drive')

drive_dataset_dir = os.path.join('/content/drive', drive_project_root)
drive_dataset_path = os.path.join('/content/drive', drive_project_root, dataset_name)

!mkdir -p $drive_dataset_dir
!test -e $drive_dataset_path \
  && echo 'Extracting the cached dataset' \
  && cp $drive_dataset_path $dataset_name \
  && tar -xzf $dataset_name

In [ ]:
# Download the datasets
!test ! -e $drive_dataset_path \
  && bash datasets.sh > /dev/null

In [ ]:
# Preprocess the dataset and bake the augmentation variant
!test ! -e $drive_dataset_path \
  && python preprocess.py --process_stanford=True \
  && python bake.py --bake_input=datasets/Stanford40/data --bake_output=datasets/Stanford40/baked --bake_optical_flow=False --augment_repeat_count=25 

In [ ]:
#@title Cache the processed dataset { display-mode: "form" }
dataset_dir = "datasets/Stanford40/baked" #@param {type:"string"}
!test ! -e $drive_dataset_path \
  && echo 'Compressing the dataset' \
  && GZIP=-9 tar --totals=USR1 -czf $dataset_name $dataset_dir \
  && echo 'Copying cache to drive' \
  && cp $dataset_name $drive_dataset_path

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_frames.py

In [ ]:
%load_ext tensorboard
# %tensorboard --logdir=logs